In [ ]:
import numpy as np
import scipy.sparse as scs
from scipy.stats import multinomial
import pathlib as pl
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import mmread

import umap as um


from os import listdir
from os.path import isfile, join
import os

import random

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

import seaborn as sns

from sklearn.preprocessing import StandardScaler

import random
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, accuracy_score, average_precision_score


import tensorflow as tf

import os
import pickle

import seaborn as sns

from sklearn.model_selection import train_test_split

from kneed import KneeLocator

from numpy.random import seed

import time


from plot_results import get_dbl_metrics
from vae import define_clust_vae
from PU import PU, epoch_PU, epoch_PU2
from classifier import define_classifier
from mk_doublets import sim_inflate, sim_avg, sim_sum
from cluster import cluster, fast_cluster
from vaeda import vaeda

import scanpy as sc
import anndata

In [ ]:
seeds = [42, 29503, 432809, 42, 132975, 9231996, 12883823, 9231996, 1234, 62938, 57203 ,109573, 23]
down_samp_seeds = [42, 29503, 432809, 4321, 132975, 9231996, 12883823, 9231996, 1234, 62938, 57203 ,109573, 23]
down_samp_seeds2 = [9231996, 1234, 62938, 57203 ,109573, 23, 42, 67, 120487, 3857, 3967, 999, 0, 275]

In [ ]:
path = '../data/'
files = [f for f in listdir(path) if isfile(join(path, f))]
files = np.sort(files)
files

In [ ]:
use_old=True
fracs = [0.95]
reps = range(10)

In [ ]:
for file in files:

    data_name = file[:-4]
    print(data_name) 
    
    real_path = '../data/mtx_files/' + data_name + '.mtx'
    ano_path  = '../data/mtx_files/' + data_name + '_anno.csv'

    dat_real = mmread(real_path)
    Xr_keep = scs.csr_matrix(dat_real).toarray().T

    #- READ IN BARCODE ANNOTATIONS
    ano = pd.read_csv(ano_path)
    true_keep = pd.factorize(ano.x)[0]
    labels_keep = ano.x
    if (labels_keep[0]=='doublet'):
        tmp = true_keep + 3
        tmp[tmp==3] = 1
        tmp[tmp==4] = 0
        true_keep = tmp

    for frac in fracs:
        for i in reps:
            
            print('data:',data_name) 
            print('frac:',frac)
            print('rep:',i)
            
            save_path = '../downsample_cells/vaeda'
            data_path = '../downsample_cells/data'
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            save_path = save_path + '/' + data_name + '_frac' + str(frac) + '_rep' + str(i) + '_'
            data_path = data_path + '/' + data_name + '_frac' + str(frac) + '_rep' + str(i) + '_'
            
            ############################################################
            ####################### DOWN SAMPLE ########################
            ############################################################

            if(pl.Path(data_path + 'ind.npy').exists()):
                
                print('exists!')
                keep_ind = np.load(data_path + 'ind.npy')
            
            else:
                print('DOES NOT EXIST')
            
                #split so that retains same fraction of real doublets
                ind = np.arange(Xr_keep.shape[0])
                sing_ind = ind[true_keep==0]
                doub_ind = ind[true_keep==1]

                num = int(len(sing_ind) * frac)
                np.random.seed(down_samp_seeds[i])
                np.random.shuffle(sing_ind)
                sing_ind = sing_ind[:num]

                num = int(len(doub_ind) * frac)
                np.random.seed(down_samp_seeds2[i])
                np.random.shuffle(doub_ind)
                doub_ind = doub_ind[:num]

                keep_ind = np.concatenate([sing_ind, doub_ind])
                np.save(data_path + 'ind.npy', keep_ind)

            Xr = Xr_keep[keep_ind,:]
            true = true_keep[keep_ind]
            labels = labels_keep[keep_ind]

            ############################################################
            ############################ PU ############################
            ############################################################

            preds, preds_on_P, calls, encoding, knn_features = vaeda(Xr, save_dir=save_path, use_old=use_old, pca_comp=30, clust_weight=20000)

            #RESULTS
            preds_sing = preds[true==0]
            preds_doub_test = preds[true==1]
            preds_doub_train = preds_on_P
            labs = ['singlet', 'actual doublet', 'simulated doublet']
            cols = np.concatenate([preds, preds_on_P])

            #SAVE SCORES
            tmp1 = np.zeros((len(preds), 3))
            df = pd.DataFrame(tmp1, columns=['ano', 'doublet_scores', 'predicted_doublets'])
            df.ano = true
            df.doublet_scores = preds
            df.predicted_doublets = calls
            if(save):
                df.to_csv(save_path + 'vaeda_scores_1.csv') 

            #PR and ROC curves
            plt.figure(4)
            res = get_dbl_metrics(true, preds)
            plt.show()
            plt.close()

            #save AUCs
            hm_pr = pd.DataFrame(np.array(res).T, index=['AUROC', 'AUPRC', 'AP']).T
            if(save):
                hm_pr.to_csv(save_path + 'scores_ROC_PR_area_ALL.csv') 




In [ ]:
print('done')